In [1]:
##importing the csv into the notebook
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
#from sklearn.cluster import KMeans
#from sklearn.cluster import kmeans_plusplus
#from sklearn.neighbors import NearestNeighbors
#from sklearn.cluster import DBSCAN
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.preprocessing import StandardScaler
import time
#from sklearn.metrics import silhouette_samples, silhouette_score
#from mpl_toolkits import mplot3d 
#from mpl_toolkits.mplot3d import Axes3D
sb.set() # set the default Seaborn style for graphics
pd.options.mode.chained_assignment = None
#scaler = MinMaxScaler() #you might need to change this according to the data set we have 

In [105]:
combined = pd.read_csv("./combined - CG CE removed.csv", index_col = [0]) #import fresh data 
combined = combined.rename(columns = {'lng_y':'lng_mrt', 'lat_y':'lat_mrt','lng_x':'lng_hdb', 'lat_x':'lat_hdb', 'distance': 'mahattan_distance', 'travel_distance':'mrt_cbd_dist', 'travel_time':'mrt_cbd_time'})
print(combined.shape)
section = {} # Create a dict, split into sections
i=0
jump=1000
## Trial debug
#while i < 50:
while i < combined.shape[0]+jump:
    section[int(i/jump)]=combined.iloc[i:i+jump-1]
    print(combined.iloc[i:i+jump-1]['postal'])
    i+=jump


(13328, 45)
0.0       189673
1.0       189673
4.0       189674
5.0       189674
8.0       460001
           ...  
1790.0    321113
1791.0    322113
1792.0    323113
1793.0    324113
1794.0    325113
Name: postal, Length: 999, dtype: object
1796.0    320119
1797.0    320012
1798.0    320120
1799.0    320121
1800.0    320122
           ...  
3019.0    460554
3021.0    460556
3023.0    461556
3025.0    460557
3027.0    460074
Name: postal, Length: 999, dtype: object
3031.0    460076
3033.0    460077
3035.0    460078
3037.0    460079
3039.0    460080
           ...  
4217.0    824105
4218.0    820106
4219.0    820106
4220.0    820106
4221.0    820106
Name: postal, Length: 999, dtype: object
4223.0    821106
4224.0    821106
4225.0    821106
4226.0    822106
4227.0    822106
           ...  
5217.0    540162
5218.0    540162
5219.0    541162
5220.0    541162
5221.0    541162
Name: postal, Length: 999, dtype: object
5223.0    542162
5224.0    542162
5225.0    543162
5226.0    543162
5227.0  

In [85]:
# Repeated (shifted above)
# combined = combined.rename(columns = {'lng_y':'lng_mrt', 'lat_y':'lat_mrt','lng_x':'lng_hdb', 'lat_x':'lat_hdb', 'distance': 'mahattan_distance', 'travel_distance':'mrt_cbd_dist', 'travel_time':'mrt_cbd_time'})

In [31]:
import requests
import json# call the OSMR API

i=1


def router(row):
    lon_1, lat_1, lon_2, lat_2 = row['lng_hdb'], row['lat_hdb'], 103.8513, 1.2830
    
    global i
    r = requests.get(f"http://router.project-osrm.org/route/v1/car/{lon_1},{lat_1};{lon_2},{lat_2}?overview=false""")# then you load the response using the json libray
    # by default you get only one alternative so you access 0-th element of the `routes`
    
    routes = json.loads(r.content)
    # print(routes)
    route_1 = routes.get("routes")[0]
    # print(f"Overall, the route data is:\n{route_1}\n")
    seconds = route_1['legs'][0]['duration']
    #print(f"Duration: {seconds}sec ({seconds/60:.2f} min )")
    print("row", i)
    print(route_1['legs'][0]['distance'])
    i += 1

    return float(route_1['legs'][0]['distance']), float(seconds/60.0)


In [106]:
i=0
for key, part in section.items():
    print(f"Current section:{key*jump}")
    print(f"Length of section: {len(section)}\n")

    #print(part)
    part[['hdb_cbd_distance', 'hdb_cbd_time']]  = part.apply(router, axis=1, result_type='expand')
    writer = pd.ExcelWriter('combined_dist.xlsx', engine = 'openpyxl',mode='a',if_sheet_exists="replace")
    part.to_excel(writer, sheet_name = f'page{key}')
    writer.close()

Current section:0
Length of section: 15

row 0
2283.5
row 1
2283.5
row 2
2888.2
row 3
2888.2
row 4
13450.9
row 5
13837.4
row 6
13953.1
row 7
13857.3
row 8
13877.3
row 9
13779.6
row 10
13728.7
row 11
14456.2
row 12
13515.6
row 13
13510.2
row 14
13482.6
row 15
13656.3
row 16
13528.4
row 17
14351
row 18
14323.6
row 19
14305.2
row 20
14337.3
row 21
14538
row 22
14599.2
row 23
14035.9
row 24
14646.8
row 25
14687.5
row 26
14387.2
row 27
13938.9
row 28
14333.2
row 29
14279.6
row 30
14196.9
row 31
13508.9
row 32
14234.6
row 33
14159.3
row 34
14487.8
row 35
14410.4
row 36
14303.8
row 37
14258.9
row 38
14212.5
row 39
14250.1
row 40
14299
row 41
14389.1
row 42
13506.4
row 43
14311
row 44
14315.4
row 45
14355.6
row 46
13702
row 47
13705.5
row 48
13734.7
row 49
13321.2
row 50
13666.6
row 51
13743
row 52
13817.3
row 53
13348.6
row 54
13526.7
row 55
13802.7
row 56
13904.3
row 57
13862
row 58
13822.2
row 59
13840.9
row 60
14197.8
row 61
13354.4
row 62
13877.3
row 63
13997.8
row 64
13855
row 65
13747.7

row 544
27494.3
row 545
27432.4
row 546
27468.2
row 547
27637.7
row 548
27450
row 549
27439.3
row 550
27403.9
row 551
27380.7
row 552
27355.9
row 553
27422.6
row 554
27421.2
row 555
27474
row 556
27398.3
row 557
27442.2
row 558
27497.7
row 559
27479
row 560
22713.3
row 561
22574.1
row 562
22557.5
row 563
22626.7
row 564
22866.6
row 565
22816
row 566
27281.4
row 567
27269.5
row 568
27265.1
row 569
27303.9
row 570
27376.7
row 571
27317.2
row 572
27342.9
row 573
27453.1
row 574
27251.6
row 575
27228.5
row 576
27228.3
row 577
27165.6
row 578
26994.9
row 579
26993.4
row 580
26964.5
row 581
27449.7
row 582
27466.6
row 583
24194.2
row 584
24247
row 585
24314.5
row 586
26910.2
row 587
27449
row 588
27318.5
row 589
27453.6
row 590
27070.3
row 591
26974.9
row 592
26974.1
row 593
27064.4
row 594
26962.7
row 595
27045.7
row 596
27083.3
row 597
27426
row 598
27106.5
row 599
27103.2
row 600
27130.6
row 601
27491.3
row 602
27488.4
row 603
27526.7
row 604
23981.7
row 605
27468.2
row 606
26552.2
row 60

row 1083
15529.7
row 1084
15583.8
row 1085
15641.8
row 1086
15526.7
row 1087
15574.2
row 1088
15793.4
row 1089
15707.8
row 1090
15642
row 1091
15624
row 1092
15548.1
row 1093
15522.2
row 1094
15550.5
row 1095
15594.5
row 1096
15551.9
row 1097
15550.8
row 1098
15651
row 1099
15596.1
row 1100
15605.8
row 1101
15476.8
row 1102
15702.1
row 1103
15639.7
row 1104
15500
row 1105
15568.3
row 1106
15549.8
row 1107
15700.3
row 1108
15406.2
row 1109
15220
row 1110
15174.2
row 1111
15187.1
row 1112
15219.8
row 1113
15138.5
row 1114
15217.6
row 1115
15429.9
row 1116
15465.5
row 1117
15361.9
row 1118
15311.4
row 1119
15264.4
row 1120
15180.3
row 1121
15227.1
row 1122
15490.2
row 1123
15431.3
row 1124
14424
row 1125
14403.3
row 1126
14532.7
row 1127
14482.5
row 1128
14445.6
row 1129
14440.5
row 1130
14429.1
row 1131
15077.9
row 1132
14417.9
row 1133
14520
row 1134
14471.8
row 1135
14456.4
row 1136
15052.9
row 1137
14550.4
row 1138
14488.3
row 1139
14486.8
row 1140
14969
row 1141
14998
row 1142
14976.

row 1572
18116.6
row 1573
18116.6
row 1574
18116.6
row 1575
18309.4
row 1576
18309.4
row 1577
18309.4
row 1578
18309.4
row 1579
18285.3
row 1580
18285.3
row 1581
18285.3
row 1582
18285.3
row 1583
18396.4
row 1584
18396.4
row 1585
18396.4
row 1586
18396.4
row 1587
18462.6
row 1588
18462.6
row 1589
18462.6
row 1590
18462.6
row 1591
18483.2
row 1592
18483.2
row 1593
18483.2
row 1594
18483.2
row 1595
18410.8
row 1596
18410.8
row 1597
18410.8
row 1598
18410.8
row 1599
18414.3
row 1600
18414.3
row 1601
18414.3
row 1602
18414.3
row 1603
18427.4
row 1604
18427.4
row 1605
18427.4
row 1606
18427.4
row 1607
19152.5
row 1608
19152.5
row 1609
19152.5
row 1610
19152.5
row 1611
18467.2
row 1612
18467.2
row 1613
18467.2
row 1614
18467.2
row 1615
18480.6
row 1616
18480.6
row 1617
18480.6
row 1618
18480.6
row 1619
18529.4
row 1620
18529.4
row 1621
18529.4
row 1622
18529.4
row 1623
18424.1
row 1624
18424.1
row 1625
18424.1
row 1626
18424.1
row 1627
18491.6
row 1628
18491.6
row 1629
18491.6
row 1630
18491

row 2062
7406.3
row 2063
7514.6
row 2064
7497.3
row 2065
7552.7
row 2066
10769.8
row 2067
10769.8
row 2068
10764.7
row 2069
10764.7
row 2070
10740.7
row 2071
10740.7
row 2072
10723.2
row 2073
10723.2
row 2074
10707.6
row 2075
10707.6
row 2076
10652.1
row 2077
10652.1
row 2078
10601.1
row 2079
10601.1
row 2080
10565.4
row 2081
10565.4
row 2082
10681.1
row 2083
10681.1
row 2084
10790.4
row 2085
10790.4
row 2086
10790.4
row 2087
10790.4
row 2088
10778.5
row 2089
10778.5
row 2090
10757.4
row 2091
10757.4
row 2092
10783.1
row 2093
10783.1
row 2094
10488.3
row 2095
10488.3
row 2096
10398.2
row 2097
10398.2
row 2098
10407.1
row 2099
10407.1
row 2100
10447.7
row 2101
10447.7
row 2102
10680.9
row 2103
10680.9
row 2104
10713.4
row 2105
10713.4
row 2106
10626.8
row 2107
10626.8
row 2108
10741.2
row 2109
10741.2
row 2110
10623.4
row 2111
10623.4
row 2112
10603.3
row 2113
10603.3
row 2114
10777.2
row 2115
10777.2
row 2116
10673.4
row 2117
10673.4
row 2118
10975.4
row 2119
10975.4
row 2120
11021.2
r

...

row 11727
22437.6
row 11728
22497.3
row 11729
22497.3
row 11730
22638.5
row 11731
22638.5
row 11732
22640
row 11733
22640
row 11734
22780.3
row 11735
22780.3
row 11736
22506.5
row 11737
22506.5
row 11738
22656.3
row 11739
22656.3
row 11740
22553.6
row 11741
22553.6
row 11742
22580.7
row 11743
22580.7
row 11744
22651.3
row 11745
22651.3
row 11746
22451.5
row 11747
22451.5
row 11748
22478.2
row 11749
22478.2
row 11750
22559.7
row 11751
22559.7
row 11752
22704.7
row 11753
22704.7
row 11754
22649.6
row 11755
22649.6
row 11756
22647.2
row 11757
22647.2
row 11758
22615.7
row 11759
22615.7
row 11760
22537.8
row 11761
22537.8
row 11762
22656.4
row 11763
22656.4
row 11764
22575.7
row 11765
22575.7
row 11766
22429.9
row 11767
22429.9
row 11768
22481
row 11769
22481
row 11770
22549.2
row 11771
22549.2
row 11772
22591.1
row 11773
22591.1
row 11774
22663.7
row 11775
22663.7
row 11776
22117.6
row 11777
22117.6
row 11778
22173.1
row 11779
22173.1
row 11780
22507.2
row 11781
22507.2
row 11782
22611.1


row 12186
18691.9
row 12187
18721.4
row 12188
18721.4
row 12189
18721.4
row 12190
18728.8
row 12191
18728.8
row 12192
18728.8
row 12193
18776.8
row 12194
18776.8
row 12195
18776.8
row 12196
18829.7
row 12197
18829.7
row 12198
18829.7
row 12199
18879.6
row 12200
18879.6
row 12201
18879.6
row 12202
19377.3
row 12203
19377.3
row 12204
19377.3
row 12205
19407.6
row 12206
19407.6
row 12207
19407.6
row 12208
19317
row 12209
19317
row 12210
19317
row 12211
19081.6
row 12212
19081.6
row 12213
19081.6
row 12214
19067
row 12215
19067
row 12216
19067
row 12217
19104
row 12218
19104
row 12219
19104
row 12220
19140.6
row 12221
19140.6
row 12222
19140.6
row 12223
19423.4
row 12224
19423.4
row 12225
19423.4
row 12226
19586.8
row 12227
19586.8
row 12228
19586.8
row 12229
19077
row 12230
19077
row 12231
19077
row 12232
19419.9
row 12233
19419.9
row 12234
19419.9
row 12235
19417.1
row 12236
19417.1
row 12237
19417.1
row 12238
19451.2
row 12239
19451.2
row 12240
19451.2
row 12241
19546.3
row 12242
19546.

row 12648
19623.9
row 12649
19563.8
row 12650
19563.8
row 12651
19563.8
row 12652
19624.4
row 12653
19624.4
row 12654
19624.4
row 12655
19637
row 12656
19637
row 12657
19637
row 12658
19669.2
row 12659
19669.2
row 12660
19669.2
row 12661
19699.1
row 12662
19699.1
row 12663
19699.1
row 12664
18907.2
row 12665
18907.2
row 12666
18907.2
row 12667
18577.2
row 12668
18577.2
row 12669
18577.2
row 12670
18884.7
row 12671
18884.7
row 12672
18884.7
row 12673
19128
row 12674
19128
row 12675
19128
row 12676
19109.6
row 12677
19109.6
row 12678
19109.6
row 12679
19023.6
row 12680
19023.6
row 12681
19023.6
row 12682
18934.9
row 12683
18934.9
row 12684
18934.9
row 12685
18937.2
row 12686
18937.2
row 12687
18937.2
row 12688
19023
row 12689
19023
row 12690
19023
row 12691
19022
row 12692
19022
row 12693
19022
row 12694
19101.1
row 12695
19101.1
row 12696
19101.1
row 12697
18843.4
row 12698
18843.4
row 12699
18843.4
row 12700
18904.5
row 12701
18904.5
row 12702
18904.5
row 12703
19054.5
row 12704
19054.

row 13107
26679.4
row 13108
27019.1
row 13109
26880.1
row 13110
26715.4
row 13111
4051.7
row 13112
4051.7
row 13113
3925.2
row 13114
3925.2
row 13115
4067.7
row 13116
4067.7
row 13117
4280.8
row 13118
4280.8
row 13119
4256.5
row 13120
4256.5
row 13121
4380.2
row 13122
4380.2
row 13123
4294.6
row 13124
4294.6
row 13125
4431.5
row 13126
4431.5
row 13127
4487.5
row 13128
4487.5
row 13129
4294.6
row 13130
4294.6
row 13131
4284.4
row 13132
4284.4
row 13133
4597.4
row 13134
4597.4
row 13135
4659.7
row 13136
4558.5
row 13137
4583.8
row 13138
4527.2
row 13139
4451.9
row 13140
5033.5
row 13141
4659.3
row 13142
4559.9
row 13143
4656.3
row 13144
4600.5
row 13145
4709.5
row 13146
4650.2
row 13147
4700.4
row 13148
4668.6
row 13149
4682
row 13150
4789
row 13151
4630.8
row 13152
4666.2
row 13153
4604.1
row 13154
4506.8
row 13155
4553.2
row 13156
4649.9
row 13157
5191.8
row 13158
5158.5
row 13159
5246.7
row 13160
4495.2
row 13161
5143.7
row 13162
5037.8
row 13163
5033.4
row 13164
5037.3
row 13165
4905

TypeError: 'NoneType' object is not subscriptable

In [83]:
#combined[['hdb_cbd_distance', 'hdb_cbd_time']]
#combined.to_csv('dist_to_mrt.csv', encoding='utf-8')
section[1][['hdb_mrt_distance', 'hdb_mrt_time']]

KeyError: "None of [Index(['hdb_mrt_distance', 'hdb_mrt_time'], dtype='object')] are in the [columns]"

In [ ]:
import requests
import json# call the OSMR API

i=1


def router2(row):
    lon_1, lat_1, lon_2, lat_2 = row['lng_hdb'], row['lat_hdb'], row['lng_mrt'], row['lat_mrt']
    
    global i
    r = requests.get(f"http://router.project-osrm.org/route/v1/car/{lon_1},{lat_1};{lon_2},{lat_2}?overview=false""")# then you load the response using the json libray
    
    routes = json.loads(r.content)
    # print(routes)
    route_1 = routes.get("routes")[0]
    # by default you get only one alternative so you access 0-th element of the `routes`

    # print(f"Overall, the route data is:\n{route_1}\n")
    seconds = route_1['legs'][0]['duration']
    #print(f"Duration: {seconds}sec ({seconds/60:.2f} min )")
    print("row", i)
    print(route_1['legs'][0]['distance'])
    i += 1
    return float(route_1['legs'][0]['distance']), float(seconds/60.0)
    


In [ ]:
section[i][['hdb_mrt_distance', 'hdb_mrt_time']]  = combined.apply(router, axis=1, result_type='expand')

i=0
jump=1000
while i < combined.shape[0]+jump:
    section[int(i/jump)]=combined.iloc[i:i+jump-1]
    print(combined.iloc[i:i+jump-1]['postal'])
    i+=jump


combined.to_excel(writer, sheet_name = 'page2')


In [ ]:
# Display rows where 'name' column matches this station specified.
combined[combined['name']=='CITY HALL MRT STATION']